# Debugging Division By Zero Errors with Avicenna

This notebook explores the use of the Avicenna tool to understand and diagnose failure scenarios that lead to a division by zero error in a calculator program. We aim to create a grammar that defines possible arithmetic expressions and utilize Avicenna's capabilities to identify expressions that could result in division by zero.

In [1]:
# Suppress logging for the notebook; uncomment the last line to disable Avicenna logs
import logging

# This will disable all logging messages
logging.disable(logging.CRITICAL)

## Defining the Grammar for Arithmetic Expressions

The grammar below specifies the structure of valid arithmetic expressions. It includes operations (addition, subtraction, multiplication, division) and numeric values, which may include whole numbers, fractions, and negative values. The grammar is designed to exhaustively explore various combinations that could potentially lead to a division by zero scenario.

In [2]:
import string
from avicenna.avicenna import Grammar, is_valid_grammar

divide_by_zero_grammar: Grammar = {
   "<start>": ["<arith_expr>"],
   "<arith_expr>": ["<arith_expr><operator><arith_expr>", "<number>", "(<arith_expr>)"],
   "<operator>": ["+", "-", "*", "/"],
   "<number>": ["<maybe_minus><non_zero_digit><maybe_digits><maybe_frac>", "<maybe_minus>0.<digits>", "<maybe_minus>0"],
   "<maybe_minus>": ["", "-"],
   "<non_zero_digit>": [str(num) for num in range(1, 10)],  # Exclude 0 from starting digits
   "<digit>": list(string.digits),
   "<maybe_digits>": ["", "<digits>"],
   "<digits>": ["<digit>", "<digit><digits>"],
   "<maybe_frac>": ["", ".<digits>"],
}

It's essential to ensure that the defined grammar is valid and can generate the intended strings without errors. The validation step confirms the grammar's structure and usability before it is used to generate test cases.

In [3]:
assert is_valid_grammar(divide_by_zero_grammar)

## Oracle Function and Initial Inputs

The oracle function evaluates arithmetic expressions to detect whether they result in a division by zero error. We define initial inputs known to either pass or trigger the division by zero error to start the diagnosis process with known outcomes. These examples help Avicenna in learning and refining its model of failure.


In [5]:
from avicenna.input.input import OracleResult

# Oracle for divide by zero
def divide_by_zero_oracle(inp: str):
   try:
       eval(str(inp))  
   except ZeroDivisionError as e:
       return OracleResult.FAILING, ZeroDivisionError
   return OracleResult.PASSING, None

# Initial inputs for Avicenna
divide_by_zero_inputs =  ['1/0', '5/(3-3)', '(2+3)/5', '7/(2*0)', '9/(0/3)']

## Initializing Avicenna for Diagnosis

An instance of Avicenna is created with the specified grammar, the oracle function, and the initial set of test inputs. We set a limit on the number of iterations Avicenna should perform, which controls the depth of analysis while trying to understand the failure scenarios.

In [6]:
from avicenna import Avicenna

# Avicenna instance
avicenna_divide_by_zero = Avicenna(
    grammar=divide_by_zero_grammar,
    oracle=divide_by_zero_oracle,
    initial_inputs=divide_by_zero_inputs,
    max_iterations=5,
)

## Diagnosis Results

After running the diagnostic process, Avicenna provides a set of constraints that describe the circumstances under which the failure occurs, along with the precision and recall of its findings. These metrics help assess the accuracy and completeness of the diagnosis.


In [7]:
# Getting the diagnosis from Avicenna
diagnosis = avicenna_divide_by_zero.explain()

In [8]:
from isla.language import ISLaUnparser

print(f"Avicenna determined the following constraints to describe the failure circumstances:\n")

print(ISLaUnparser(diagnosis[0]).unparse())
print(f"Avicenna calculated a precision of {diagnosis[1]*100:.2f}% and a recall of {diagnosis[2]*100:.2f}%", end="\n\n")

Avicenna determined the following constraints to describe the failure circumstances:

exists <operator> elem in start:
  (= elem "/")
Avicenna calculated a precision of 85.29% and a recall of 100.00%



This constraint specifies that a failure (division by zero error) is predicted to occur if there exists an operator element (\<operator>) in the starting expression (\<start>) which is specifically a division operator ("/"). In simpler terms, Avicenna has learned that the presence of a division operator in an arithmetic expression is necessary for a division by zero error to potentially occur.

#### Limitations

The diagnosis provided by Avicenna, stating the existence of an operator "/" in the expression as a key to predicting a division by zero, and the noted precision and recall metrics offer insights into its performance. However, this diagnosis isn't perfect for identifying all arithmetic expressions that can cause a division by zero for several reasons:

The constraint 
```
exists <operator> elem in start:
    (= elem "/")
```
simplifies the problem by stating that the presence of a division operator "/" anywhere in the expression is a significant indicator of a potential division by zero. This approach is too general because:

- **Not all divisions lead to zero**: Just because an expression contains a division does not necessarily mean it will result in a division by zero. The diagnosis doesn't consider the value of the denominator, which is crucial.
- **False positives**: This leads to many false positives, where divisions are normal and valid (e.g., (2+3)/5), which explains why the precision is less than 100%.
- **Position and role of zeros**: For a division by zero to occur, the division operator must be immediately followed by a zero or an expression that evaluates to zero. The diagnosis doesn’t specify this.
- **Complex expressions**: Expressions like 5/(3-3) or 7/(2*0) involve more complex scenarios where the denominator results in zero due to the operation, not just the presence of a zero or division operator.

For the diagnosis to be perfect, Avicenna would need to more accurately identify only those scenarios where the denominator evaluates to zero, without overgeneralizing to all uses of the division operator. Improving the model might involve refining the grammar used to generate test cases, providing more diverse training inputs, and possibly enhancing the model's understanding of complex arithmetic expressions. The goal would be to maintain the high recall while increasing precision, thus reducing false positives.

## Evaluation of the Learned Constraint

In [9]:
from debugging_framework.fuzzingbook.fuzzer import GrammarFuzzer
from debugging_framework.input.input import Input

grammar = divide_by_zero_grammar

evaluation_data_set = set()
fuzzer = GrammarFuzzer(grammar)

for _ in range(1000):
    tree = fuzzer.fuzz()
    evaluation_data_set.add(Input.from_str(grammar=grammar, input_string=tree))

print(f"Generated {len(evaluation_data_set)} inputs for evaluation!")

Generated 764 inputs for evaluation!


In [10]:
from debugging_framework.input.input import OracleResult

oracle = divide_by_zero_oracle
failing = set()
passing = set()

for inp in evaluation_data_set:
    oracle_result, exception = oracle(inp)

    if oracle_result == OracleResult.FAILING:
        failing.add(inp)
    elif oracle_result == OracleResult.PASSING:
        passing.add(inp)

print(f"Generated {len(passing)} passing inputs for evaluation!")
print(f"Generated {len(failing)} passing inputs for evaluation!")

Generated 569 passing inputs for evaluation!
Generated 195 passing inputs for evaluation!


In [11]:
from isla.evaluator import evaluate

eval_results_passing = []
for inp in list(passing):
    eval_results_passing.append(bool(evaluate(diagnosis[0], inp.tree, grammar)))

eval_results_failing = []
for inp in list(failing):
    eval_results_failing.append(bool(evaluate(diagnosis[0], inp.tree, grammar)))

In [12]:
tp = sum(int(entry) for entry in eval_results_failing)
fn = len(eval_results_failing) -tp
fp = sum(int(entry) for entry in eval_results_passing)

precision = tp / (tp + fp)
recall = tp / (tp + fn)

print(f"The Diagnosis achieved a Precision of {precision*100:.2f}% and a Recall of {recall*100:.2f}%")

The Diagnosis achieved a Precision of 72.49% and a Recall of 100.00%


In [13]:
tp = sum(int(entry) for entry in list(eval_results_failing))
fn = len(eval_results_failing) -tp
fp = sum(int(entry) for entry in eval_results_passing)

precision = tp / (tp + fp)
recall = tp / (tp + fn)

print(f"The Diagnosis achieved a Precision of {precision*100:.2f}% and a Recall of {recall*100:.2f}%")

The Diagnosis achieved a Precision of 72.49% and a Recall of 100.00%
